- Query: `"#dudaRAE"` per un seguit de franges temporals, fent servir `since:` i `until:`

- Es crea un diccionari de respostes de l'API segons aquestes dates.

- Processat de dades i obtenció de respostes als tweets obtinguts

Nota: projecte vell, és possible que falli alguna cosa

In [ ]:
import asyncio
import pickle
from datetime import datetime, timedelta

import pandas as pd
from tqdm import tqdm
from twitter_scrape_utils import setup_accounts, tweets2df
from twscrape import API, gather
from twscrape.logger import set_log_level

set_log_level("DEBUG")
api = API()  # initialize the API with default account database `accounts.db`

pd.set_option("display.max_colwidth", None)

In [ ]:
await setup_accounts(api)

- màxim de 3 tweets per dia pel 2023 -> l'API s'ho passa pel forro
- màxim de 2 respostes (que siguin del compte @RAEinforma) 

In [ ]:
from datetime import datetime, timedelta

start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

date_queries = {}
current_date = start_date
while current_date <= end_date:
    since_date = current_date.strftime("%Y-%m-%d")
    until_date = (current_date + timedelta(days=1)).strftime("%Y-%m-%d")
    query = f"#dudaRAE since:{since_date} until:{until_date}"
    date_queries[since_date] = query

    current_date += timedelta(days=1)

In [ ]:
# with open("tweets.pkl", "wb") as f:
#     pickle.dump(tweets, f)

In [ ]:
tweets = {}
with open("tweets.pkl", "rb") as f:
    tweets = pickle.load(f)

In [ ]:
for date, query in date_queries.items():
    if date in tweets:
        continue
    tweets[date] = await gather(api.search(query, limit=3))

In [ ]:
def parse_tweet(tweet):
    return {
        "id": tweet.id,
        "username": tweet.user.username,
        "content": tweet.rawContent,
        "created_at": tweet.date,
        "likes": tweet.likeCount,
        "retweets": tweet.retweetCount,
        "replies": tweet.replyCount,
        "parent": tweet.inReplyToTweetId,
        "photos": tweet.media.photos,
        "links": tweet.links,
        "location": tweet.user.location,
    }

In [ ]:
dfs = {k: tweets2df(v, parse_tweet) for k, v in tweets.items()}

df = pd.concat(list(dfs.values()))
df["dia"] = df.created_at.astype(str).str[:10]
df["mes"] = df.created_at.astype(str).str[:7]
df["any"] = df.created_at.astype(str).str[:4]
df["hora"] = df.created_at.astype(str).str[11:19]
df.location = df.location.str.lower().str.strip()

df = df[df["any"] == "2023"]

In [ ]:
df.shape, df[df.replies != 0].shape, df[df.replies == 0].shape

In [ ]:
df.location.value_counts()

Filtro els textos que no citen el compte de la RAE, els que no tenen respostes, i els que en tenen cinc o més.
A més, trec els que tenen fotos o enllaços.
Finalment, n'agafo 50 de cada mes aleatòriament.

In [ ]:
filtered = (
    df[
        (df.content.str.contains("@RAEinforma"))
        # & (df.replies == 0)
        & (df.replies > 0)
        & (df.replies < 5)
        & (~df.photos.astype(bool))
        & (~df.links.astype(bool))
    ]
    .groupby("mes", group_keys=False)[
        [
            "id",
            "content",
            "created_at",
            "dia",
            "mes",
            "hora",
        ]
    ]
    .apply(lambda x: x.sample(5, random_state=42))
)

In [ ]:
# filtered[["id", "content", "dia", "mes", "hora"]].to_excel("sin_respuesta.xlsx", index=False)

In [ ]:
# Si ens interessen les repostes
async def get_replies(ID, limit=2):
    return await gather(api.tweet_replies(ID, limit=limit))

In [ ]:
# Si ens interessen els anteriors
async def get_id(ID):
    tweet = await api.tweet_details(ID)
    return parse_tweet(tweet)


# parent = await get_id(df.iloc[0].parent)

In [ ]:
filtered["no_resposta"] = 0
filtered["varies_respostes"] = 0

replies = []

In [ ]:
with open("replies.pkl", "wb") as f:
    pickle.dump(replies, f)

In [ ]:
for i, ID in tqdm(enumerate(filtered.id)):
    rep = await get_replies(ID)
    df_rep = tweets2df(rep).query("username == 'RAEinforma'")
    if len(df_rep) == 0:
        filtered.loc[filtered.id == ID, "no_resposta"] = 1
    elif len(df_rep) > 1:
        filtered.loc[filtered.id == ID, "varies_respostes"] = 1
        replies.append(df_rep)
    else:
        replies.append(df_rep)
        if df_rep.iloc[0].replies > 0:
            rep2 = await get_replies(df_rep.id.iloc[0])
            if rep2:  # even though I already filtered for replies, the output may be empty
                df_rep2 = tweets2df(rep2).query("username == 'RAEinforma'")
                if len(df_rep2) > 1:
                    replies.append(df_rep)

In [ ]:
df_replies = pd.concat(replies)

In [ ]:
# duplicats perquè vaig repetir el loop
df_replies[["parent", "content"]].shape

In [ ]:
df_replies = df_replies[["parent", "content"]].drop_duplicates()
df_replies.shape

Tots tenen una única resposta

In [ ]:
df_replies.parent.nunique()

In [ ]:
# fals, estaven = 0 els 2 quan vaig tirar el loop
filtered.no_resposta.sum(), filtered.varies_respostes.sum()

In [ ]:
corpus = filtered.merge(df_replies[["parent", "content"]], how="left", left_on="id", right_on="parent").rename(
    columns={"content_x": "pregunta", "content_y": "resposta"}
)[["id", "pregunta", "resposta", "dia", "mes", "hora"]]
corpus.to_csv("RAEconsultas_600.csv", index=False)
corpus.to_excel("RAEconsultas_600.xlsx", index=False)

In [ ]:
corpus.mes.value_counts()